In [18]:
import nltk
import random
from unidecode import unidecode
import pickle

In [19]:
short_pos=open("positive.txt","r").read()
short_neg=open("negative.txt","r").read()

short_pos = short_pos.decode('unicode_escape').encode('ascii','ignore')
short_neg = short_neg.decode('unicode_escape').encode('ascii','ignore')

In [20]:
documents=[]    # for getting all the reviews in the form (review,pos or neg)
all_words=[]    # our BAG of words

In [21]:
# Now getting all words in the documents using word tokienizer
from nltk.tokenize import word_tokenize

allowed_word_types=["J"]  # pos tagging, j means adjactive


for p in short_pos.split('\n'):
    documents.append( (p, "pos") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())
            
for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:  # so that only adjactives will be in our bag of words
            all_words.append(w[0].lower())

save_documents = open("pickled_algo/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

In [22]:
all_words=nltk.FreqDist(all_words)

In [23]:
word_features = list(all_words.keys())[:5000] #getting top 5000 words


##---- pickle

save_word_features = open("pickled_algo/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

##--------------------------##

# find feature function checks if a word that is present in our word_features(BOW for top 5000) is also present in 
# the document given
# after training the final model, for using this module for new document 
# we will first call this fuction, which will give back word_features.
    
def find_features(document):
    words=word_tokenize(document)
    features={}
    for w in word_features:
        features[w]=(w in words)

    return features      ## map of 5000 words,true if present in document and false otherwise

In [24]:
#feature_Set=pickle.load( open( "feature.pickle", "rb" ) )

feature_Set=[]      ## training and testing data
                    ## 5000 word fetures ---> Pos||Neg    
                    # 1 2 3 4 5 6 ......... 5000  POS||NEG


for rev,category in documents: # documents contain reviews by line and their tags(pos or neg)
    feature_Set.append((find_features(rev),category))


random.shuffle(feature_Set)

## pickle
save_feature=open("pickled_algo/feature.pickle","wb")
pickle.dump(feature_Set,save_feature)
save_feature.close()

In [25]:
train_set=feature_Set[10000:]
test_set=feature_Set[:10000]

In [26]:
classifier=nltk.NaiveBayesClassifier.train(train_set)

#pickle
save_feature=open("pickled_algo/originalnb.pickle","wb")
pickle.dump(classifier,save_feature)
save_feature.close()

In [27]:
print ("Naive Bayes  Algo accuracy percentage :",(nltk.classify.accuracy(classifier,test_set))*100)

('Naive Bayes  Algo accuracy percentage :', 60.209999999999994)


In [28]:
classifier.show_most_informative_features(15)

Most Informative Features
                    well = True              pos : neg    =      7.9 : 1.0
                    best = True              pos : neg    =      4.8 : 1.0
                     bad = True              neg : pos    =      4.1 : 1.0
                romantic = True              pos : neg    =      3.8 : 1.0
               cinematic = True              pos : neg    =      3.8 : 1.0
                    dull = True              neg : pos    =      3.6 : 1.0
                   debut = True              pos : neg    =      3.1 : 1.0
                   sharp = True              pos : neg    =      3.1 : 1.0
                  female = True              neg : pos    =      2.9 : 1.0
                      de = True              neg : pos    =      2.9 : 1.0
                 earnest = True              neg : pos    =      2.9 : 1.0
                   quite = True              neg : pos    =      2.9 : 1.0
                     big = True              pos : neg    =      2.7 : 1.0

In [29]:
#using sklearn
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC,LinearSVC,NuSVC

In [30]:
nb={}
linear={}
svm={}
clas={1:[nb,2],2:[linear,2],3:[svm,2]}
nb[1]=(MultinomialNB(),"MultinomialNB")
nb[2]=(BernoulliNB(),"BernoulliNB")
linear[1]=(LogisticRegression(),"LogisticRegression")
linear[2]=(SGDClassifier(),"SGDClassifier")
svm[1]=(LinearSVC(),"LinearSVC")
svm[2]=(NuSVC(),"NuSVC")

In [31]:
fin_classi=[]
for i in range(1,4):
    for j in range(1,clas[i][1]+1):
        dic=clas[i][0]
        classif=SklearnClassifier(dic[j][0])
        
        classif.train(train_set)
        
        #pickle
        k=dic[j][1]+'.pickle'
        save_feature=open(k,"wb")
        pickle.dump(feature_Set,save_feature)
        save_feature.close()
        ###
        
        fin_classi.append(classif)
        print ("classifier {} ".format(dic[j][1]),(nltk.classify.accuracy(classif,test_set))*100)

('classifier MultinomialNB ', 60.309999999999995)
('classifier BernoulliNB ', 60.17)
('classifier LogisticRegression ', 58.64)
('classifier SGDClassifier ', 58.699999999999996)
('classifier LinearSVC ', 59.25)
('classifier NuSVC ', 57.49999999999999)


In [32]:
## function for voting between the classifier to get best result  also to get confidence level.
# should be used after training  individual models

from nltk.classify import ClassifierI
from statistics import mode
class Vclassify(ClassifierI):
    def __init__(self,classifiers):
        self._classifiers=classifiers
    def classify(self,features):
        votes=[]
        for c in self._classifiers:
            
            v=c.classify(features)
            votes.append(v)
      
        return mode(votes)
    def confidence(self,features):
        votes=[]
        for c in self._classifiers:
            v=c.classify(features)
            votes.append(v)
        choice_votes=votes.count(mode(votes))
        conf=choice_votes/len(votes)
        return conf

In [33]:
fin_classi.append(classifier) #should have odd number of classifiers,so added our original NB classifier from nltk

votes_classifier=Vclassify(fin_classi)
print ("classifier",(nltk.classify.accuracy(votes_classifier,test_set))*100)

('classifier', 59.870000000000005)
